# human pose estimation - 2D 영상 내에서 (x, y) pose (관절) 의 위치를 찾는 방법

- Human pose estimation은 크게 2D와 3D 로 나누어지는데 2D HPE 는 2D 이미지에서 (x, y) 2차원 좌표들을 찾아내고, 3D HPE 는 2D 이미지에서 (x, y, z) 3차원 좌표들을 찾아내는 기술임.
- 사람의 몸은 3D 환경에서 제약이 있기 때문에 이러한 조건을 이용해서 2D에서 3D이미지를 복원함.

-  face landmark 와 비슷하지만 face landmark 는 물리적으로 거의 고정되어있지만 human pose 는 팔, 다리가 상대적으로 넓은 범위와 자유도를 갖는다는 것을 고려해야 함.
- 자유도가 높다는 것은 데이터 분포를 특정하기 어렵다고 표현할 수 있음.
- 이는 학습에 더 많은 데이터가 필요하고 더 복잡한 모델을 사용해야 한다는 것을 의미함.
- 가장 처음 접근할 수 있는 방법은 Top-down 방법과 Bottom-up 방법임.

## Top-down
- 모든 사람의 정확한 keypoint 를 찾기 위해 object detection 을 사용함.
- crop 한 이미지 내에서 keypoint 를 찾아내는 방법으로 표현함.
- detector가 선행되어야 하고 모든 사람마다 알고리즘을 적용해야 하기 때문에 사람이 많이 등장할 때는 느리다는 단점이 있음.
- 핸드폰 카메라로 찍는 인물들은 대체로 소수로 등장하기 때문에 top-down 방식을 이용해도 큰 속도 저하 없이 사용할 수 있을 것이라 생각.

## Bottom-up
- detector가 없고 keypoint 를 먼저 검출함.
- 예를 들어 손목에 해당하는 모든 점들을 검출할 때 한 사람에 해당하는 keypoint 를 clustering 함.
- detector 가 없기 때문에 다수의 사람이 영상에 등장하더라도 속도 저하가 크지 않지만 top down 방식에 비해 keypoint 검출 범위가 넓어 성능이 떨어진다는 단점이 있음.

## human keypoint detection
- human pose estimation 은 keypoint 의 localization 문제를 푼다는 점에서 비슷함.
- 손목, 팔꿈치 등의 joint keypoint 정보는 얼굴의 keypoint 보다 훨씬 다양한 위치와 변화를 가짐.
- 다루는 데이터가 2D 이미지이기 떄문에 인체의 변형 가능 문제를 해결하기 위한 방법으로 Deformable part models 방법이 있었음.
- 이는 부분(part) 들의 complex joint relationship 의 mixture model 로 keypoint 를 표현하는 방법을 이용했지만 성능은 사람들의 기대에 미치지 못함.

## DeepPose
- graphical tree model 은 같은 이미지에 두 번 연산을 하는 등 연산 효율이 떨어지는 점과 그에 비해서도 부족한 성능이 문제점으로 인식되어 옴.
- Toshev and Szegedy 는 처음으로 딥러닝 기반 keypoint localization 모델을 제안함.
- 초기의 pose estimation 모델은 x,y 좌표를 직접적으로 예측하는 position regression 문제로 인식했으며 human detection 을 통한 crop 된 사람 이미지를 이용해서 딥러닝 모델에 입력하고 (x,y) 좌표를 출력하도록 만듦.
- DeepPose는 L2 Loss 함수를 사용함.
- 전반적으로 높은 성능을 나타내긴 하지만 비약적으로 성능 향상을 시킨건 아님.
- DeepPose 의 기여는 SOTA에 가까운 성능을 내면서도 딥러닝을 적용한 첫 번째 사례임.

## Efficient Object Localization Using Convolutional Network
- Tompson 이 제안한 Efficient object localization 방법임.
- keypoint의 위치를 직접 예측하기보다 keypoint가 존재할 확률 분포를 학습하게 하고자 함.
- 눈을 찍고 싶다면 눈을 중심으로 "어떤 분포" 의 에러가 더해져서 저장되는 것.
- 자연상태에서 일어나는 확률 분포는 가우시안 분포일 가능성이 크기 때문에 label 을 (x,y) 좌표에서 (x,y) 를 중심으로 하는 heatmap 으로 변환하고 딥러닝 모델은 이 heatmap 을 학습하게 됨.
- keypoint 가 존재할 확률을 학습한 이후로 성능이 비약적으로 향상됨.
- [heatmap 학습 뿐 아니라 모델 개선도 함](https://arxiv.org/pdf/1411.4280.pdf)
  - coarse model 에서 32x32 heatmap 을 대략적으로 추출한 후 multi resolution 입력을 coarse heatmap 기준으로 crop 한 뒤 fine model 에서 refinement 를 수행함.
  - coarse model 과 fine model 이 같은 모델이며 weight 를 공유합니다. 목적이 같기 때문에 빠른 학습이 가능하고 메모리, 저장공간을 효율적으로 사용할 수 있기 때문에 weight sharing 이 적용됨.

## Convolutional Pose Machines
- CPM 은 end-to-end 로 학습할 수 있는 모델을 제안함.
- Multi-stage 방법을 사용하기 때문에 end-to-end 로 학습이 가능하더라도 그대로 학습하는 경우는 높은 성능을 달성하기 어려움.
- 따라서 stage 단위로 pretraining 을 한 후 다시 하나의 모델로 합쳐서 학습함.

## Stacked Hourglass Network
- [ECCV16 에서는 DeepPose 이후 랜드마크라고 불릴만한 논문이 제안 됨.](https://arxiv.org/pdf/1603.06937.pdf)

### Hourglass
- Stacked Hourglass Network 의 기본 구조는 모래시계 같은 모양임.
- Conv layer 와 pooling 으로 이미지(또는 feature) 를 인코딩 하고 upsampling layer 를 통해 feature map 의 크기를 키우는 방향으로 decoding함.
- feature map 크기가 작아졌다 커지는 구조여서 hourglass 라고 표현함.
- pooling으로 image의 global feature를 찾고 upsampling으로 local feature를 고려하는 아이디어가 hourglass의 핵심 novelty임.
- Hourglass는 간단한 구조를 여러 층으로 쌓아올려서 (stacked) human pose estimation의 성능을 향상시킴.

## SimpleBaseline
- [simpleBaseline](https://arxiv.org/pdf/1804.06208.pdf) 은 아주 간단한 encoder-decoder 구조를 설계함.
- resnet 50 만 사용한 간단한 구조가 hourglass 와 같은 SOTA(state-of-the-art) 를 이긴 것.

## Deep High-Resolution Network (HRNet)
- 개발된 이후 현재까지도 SOTA 에 가까운 성능을 보일 정도로 성능이 좋은 알고리즘임.
- Stacked hourglass, Casecaded pyramid network 등은 multi-stage 구조로 이루어져 있어서 학습 & 추론 속도가 느리다는 큰 단점이 있지만 하이퍼파라미터를 최적화할 경우 1-stage 방법보다 성능이 좋음.
- Simplebaseline 과 HRNet은 간단함을 추구하는 만큼 1-stage 를 고수함. 때문에 구조도 간결하고 사용하기도 쉬움.
- Simplebaseline은 high resolution → low resolution 인 encoder 와 low → high 인 decoder 구조로 이루어진 다른 알고리즘들과 공통점이 있음.
  - down sample layer를 만들고 작아진 layer feautre 정보를 다시 up sampling해서 원본 해상도 크기에 적용하는 모델을 제안함. 
  - 1-stage로 동작하기 때문에 전체 flow를 보면 엄청 간단함.
- Hourglass 는 encoder 와 decoder 의 비율이 거의 비슷함(대칭적임). 반면 Simplebaseline 은 encoder 가 무겁고 (resnet50 등 backbone 사용) decoder 는 가벼운 모델을 사용함.
- pooling(strided conv) 할 때 소실되는 정보를 high level layer에서 사용해서 detail한 정보를 학습하기 위해 skip connection을 사용함.
- [PyTorch 코드](https://github.com/leoxiaobin/deep-high-resolution-net.pytorch)

## Pose Estimation

### SimpleBaseline 구조
- conv model 로 resnet을 사용함.
- deconv module 은 deconv-bn-relu 이 단계가 3개로 이루어져 있음.
- deconv 는 256 filter size, 4x4 kernel, stride 2 로 2배씩 feature map이 커짐.
- 마지막 출력 레이어는 k 개의 1x1 conv layer로 구성됨.
- [파이토치 코드](https://github.com/Microsoft/human-pose-estimation.pytorch)
  - [모델 부분 코드](https://github.com/microsoft/human-pose-estimation.pytorch/blob/master/lib/models/pose_resnet.py)
  - torch.nn 으로 keras.layers 와 같이 딥러닝 모델 구성에 필요한 도구들이 정의되어 있음.
  - BasicBlock 이라는 클래스가 보이네요. keras.models 로 model 을 선언하는 것과 비슷함.
  - pytorch model 에서는 사용된 layer 를 forward 함수를 통해 computational graph 를 그려줌.

## SimpleBaseline - tf2

In [1]:
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet')

94773248/94765736 [==============================] - 2s 0us/step


In [2]:
#deconv module
upconv1 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn1 = tf.keras.layers.BatchNormalization()
relu1 = tf.keras.layers.ReLU()
upconv2 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn2 = tf.keras.layers.BatchNormalization()
relu2 = tf.keras.layers.ReLU()
upconv3 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same')
bn3 = tf.keras.layers.BatchNormalization()
relu3 = tf.keras.layers.ReLU()

In [3]:
#deconv module에서 중복을 제거
def _make_deconv_layer(num_deconv_layers):
    seq_model = tf.keras.models.Sequential()
    for i in range(num_deconv_layers):
        seq_model.add(tf.keras.layers.Conv2DTranspose(256, kernel_size=(4,4), strides=(2,2), padding='same'))
        seq_model.add(tf.keras.layers.BatchNormalization())
        seq_model.add(tf.keras.layers.ReLU())
    return seq_model

upconv = _make_deconv_layer(3)

In [4]:
final_layer = tf.keras.layers.Conv2D(17, kernel_size=(1,1), padding='same')

In [5]:
inputs = keras.Input(shape=(256, 192, 3))
x = resnet(inputs)
x = upconv(x)
out = final_layer(x)
model = keras.Model(inputs, out)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 192, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
sequential (Sequential)      (None, 64, 48, 256)       10489600  
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 48, 17)        4369      
Total params: 34,081,681
Trainable params: 34,027,025
Non-trainable params: 54,656
_________________________________________________________________


In [6]:
np_input = np.zeros((1,256,192,3), dtype=np.float32)
tf_input = tf.convert_to_tensor(np_input, dtype=tf.float32)
print('input shape')
print (tf_input.shape)
print('\n')

tf_output = model(tf_input)
print('output shape')
print (tf_output.shape)
print (tf_output[0,:10,:10,:10])

input shape
(1, 256, 192, 3)


output shape
(1, 64, 48, 17)
tf.Tensor(
[[[ 5.09079220e-03 -1.11279625e-03 -3.27316299e-03  1.15036368e-02
    3.36921611e-03 -6.10918738e-03  5.26093692e-03  1.62721772e-04
   -9.38179623e-03  8.90806876e-03]
  [-3.69718345e-03  2.32853368e-02  1.73550677e-02  2.52295919e-02
    1.53910229e-03 -1.50691206e-02 -1.38223385e-02 -2.76123779e-03
   -6.78293454e-03  7.80668156e-03]
  [ 2.91822925e-02  4.86187777e-03  1.15750507e-02 -3.12774465e-03
   -7.43424706e-03 -4.67731012e-03 -1.19440220e-02  3.12444055e-03
    7.20588351e-03  2.72795353e-02]
  [ 6.85881870e-03 -8.50130711e-03  2.79750358e-02  1.22712916e-02
   -8.40336643e-03 -1.48203904e-02 -2.18606535e-02  1.88235808e-02
    1.18141752e-02  4.17038985e-02]
  [ 1.36854285e-02  4.74119186e-03  3.39770829e-03  1.78719740e-02
   -8.54398124e-03  7.84097426e-03  3.16361617e-03  1.09176151e-02
   -1.67787187e-02  2.24507563e-02]
  [ 8.16416694e-04  1.13574392e-03 -1.82425359e-03  2.57616788e-02
   -6.498514